## String polytopes

INPUT: 
- ``word`` -- a reduced decomposition of the longest element in $S_{n+1}$ (in fact, len(word) = n(n+1)/2)
- ``weight`` -- weight which should be an integral vector in $\mathbb{Z}^n$

OUTPUT:
- string polytope -- a Polyhedron of dimension = len(word) = n(n+1)/2

References </br>
Theorem 3.14 in A. Berenstein, A. Zelevinsky, Tensor product multiplicities, canonical bases and totally positive varieties. Invent. math. 143, 77–128 (2001).

In [1]:
def string_polytope(word,weight):
    ## word should be a sequence of elements in [1,...,n] 
    ## len(weight) = n
    n = max(word)
    
    if not set(word).issubset(set(range(1,n+1))):
        raise ValueError("'word' must be a sequence of elements in [1,...,n].") 
    
    if not word in Permutation(range(n+1,0,-1)).reduced_words():
        raise ValueError("'word' must be a reduced decomposition of the longest element [n, n-1, ..., 1] of S_{n+1}.")

    if len(weight) != n or not all([isinstance(wt, (Integer, int)) for wt in weight]) or not all([we >= 0 for we in weight]): 
        raise ValueError("'weight' must be a nonnegative integer vector of length n.")
    
    Sineq = scone(n, word)
    Lineq = lcone(word)
    return Polyhedron(ieqs = [[0]+sineq  for sineq in Sineq] +[[weight[word[jj]-1]]+Lineq[jj] for jj in range(len(Lineq))])
    
    
def lcone(word):    
    Lmatrix = []
    length = len(word)
    for ii in range(length):
        jjrow = [0]*ii + [-1]
        for jj in range(ii+1,length):
            if abs(word[ii]-word[jj]) == 1:
                jjrow += [1]
            elif word[ii] == word[jj]:
                jjrow += [-2]
            else:
                jjrow += [0]
        Lmatrix += [jjrow]        
    return Lmatrix
    
def si(vec,ii): ## Here, ii is 1-based.
    if ii == 0:  
        raise ValueError("'ii' must be greater than 0") 
    if ii > len(vec)-1:
        raise ValueError("'ii' must be less than len(vec).")          
    vec[ii-1], vec[ii] = vec[ii], vec[ii-1]
    return vec 

## root vectors H(n,ii) = e[ii] - e[ii+1] in R^{n+1} r
## 1-based, 1 <= ii <= n
def H(n,ii): 
    if ii < 1 or ii > n:  
        raise ValueError("'ii' must be an integer between 1 and n.") 
    temp = [0]*(n+1)
    temp[ii-1] = 1
    temp[ii] = -1
    
    return temp 


## w_vectors pi(n,ii) = e[0] + ... + e[ii] in R^{n+1}
## 1-based, 1 <= ii <= n
def pi(n,ii):
    if ii < 1 or ii > n:  
        raise ValueError("'ii' must be an integer between 1 and n.") 
    temp = [0]*(n+1)
    for kk in [0..ii-1]:
        temp[kk] = 1

    return temp



# -------------------------------------------
# Closed-form u(i) and verification
# -------------------------------------------

def u_i_closed_form(n, i):
    """
    Closed-form one-line formula for u(i) in S_{n+1}:
    [i+1, i+2, ..., n, 1, n+1, 2, 3, ..., i]
    """
    if not (isinstance(n, (Integer, int)) and n > 0): 
        raise ValueError("'n' must be a positive integer.") 

    if i not in range(1,n+1):
        raise ValueError("'i' must be in {1,...,n}") 
    return list(range(i+1, n+1)) + [1, n+1] + list(range(2, i+1))

# -------------------------------------------
# Enumerate all reduced words (as tuples of simple indices)
# -------------------------------------------

def u_decomps(n): 
    if not (isinstance(n, (Integer, int)) and n > 0): 
        raise ValueError("'n' must be a positive integer.")     
    return [Permutation(u_i_closed_form(n,ii)).reduced_words() for ii in range(1,n+1)]

def refl(words,vec): 
    temp = vec[:]
    l = len(words)
    for ii in range(l):
        temp = si(temp,words[l-ii-1])
    result = temp
    
    return result

def subfinder(mylist, pattern):
    l_pattern = len(pattern)
    l_mylist = len(mylist)
    all_combi = Combinations(range(l_mylist),l_pattern).list()
    
    good_combi = []

    for combi in all_combi:
        if [mylist[com] for com in combi] == pattern:
            good_combi += [combi]
    return good_combi

## elementary basis vectors in n*(n+1)/2 
def tk(n):
    vec = []
    N = n*(n+1)//2 
    for kk in range(N):
        vec += [[0]*(kk) + [1] + [0]*(N-kk-1)]    
    return vec 

def scone(n, word):
    Smatrix = []
    length = len(word)
    u_decompositions = u_decomps(n)
    tk_vector = tk(n)
    for ii in range(1,n+1): 
        for u_decom in u_decompositions[ii-1]: 
            subwords = subfinder(word, u_decom)
            for subword in subwords:
                subword.insert(0,-1)
                subword.append(length)
                temp = vector([0]*length)
                for j in range(0,len(subword)-1):
                    for kk in range(subword[j]+1,subword[j+1]):
                    # root_vector = H(n,word[kk]-1)
                    # weight_vector = pi(n,u_decom)
                        temp = temp + vector(refl([word[subword[ll]] for ll in range(1,j+1)],H(n,word[kk])))*vector(pi(n,ii))*vector(tk_vector[kk])
                Smatrix += [list(temp)]
    return Smatrix

## Fujita--Oya polytopes

INPUT: 
- ``word`` -- a reduced decomposition of the longest element in $S_{n+1}$ (in fact, len(word) = n(n+1)/2)
- ``weight`` -- weight which should be an integral vector in $\mathbb{Z}^n$

OUTPUT:
- Fujita--Oya polytope -- a Polyhedron of dimension = len(word) = n(n+1)/2

References </br>
N. Fujita and H. Oya_Newton-Okounkov polytopes of Schubert varieties arising from cluster structures,
Trans. Amer. Math. Soc. Series B 12, 910–973 (2025).

In [2]:
def FO_polytope(seq_i, weight = None):
    n = max(seq_i)

    if weight == None:
        weight = [2]*n
    return transf_Mi(seq_i, string_polytope(seq_i, weight))

## Consider a transformation after applying Mi 
## It may change the volume because we set the distance between the origin and each facet is 1. 
def transf_Mi(seq_i, P):
    n = max(seq_i)
    N = len(seq_i)
    
    P_ineq = P.Hrepresentation()
    P_matrix = matrix([-f.outer_normal() for f in P_ineq]).transpose() 

    P_transform = Mi(n, seq_i)*P_matrix

    return Polyhedron(ieqs = [[1]+list(vec) for vec in list(P_transform.transpose())])

## Mi produces the matrix in Fujita-Oya
def Mi(n,seq_i):
    ## for s-th row, the k-th entry is 
    ## H(seq_i[k]) \cdot s_{seq_i[k+1]} s_{seq_i[k+2]} ... s_{seq_i[s]} pi(n,seq_i[s])

    N = len(seq_i)
    Mi = matrix(N)

    for ss in [0.. N-1]:
        for kk in [0.. ss]:
            H_vector = H(n,seq_i[kk])
            pi_initial = [pi(n,seq_i[ss])]
            for jj in list(range(ss,kk,-1)):
                pi_initial.append(si(pi_initial[-1], seq_i[jj]))
            Mi[ss,kk] = sum([H_vector[a]*pi_initial[-1][a] for a in list(range(n))])
    return Mi



## Triangular quiver

The following defines a matrix for the triangular quiver having $n$ vertices on each side. 
#### INPUT: 
- ``$n$`` -- An integer indicating the number of vertices on each side. Hence, the total number of vertices is $n(n+1)/2$.
- ``frozen`` -- A list which is a sublist of $[1,...,n(n+1)/2]$ indicating the frozen vertices. (default = emptyset)
There are $i$ vertices on each $i$th column for $i=1,\dots,n$.

#### Output
ClusterQuiver: A triangular quiver having $n$ vertices on each side. 

In [3]:
def triangular_quiver(n, F = []):
    """
    Label order:
      (1,1), (1,2), (2,2), ..., (1,n), (2,n), ..., (n,n)
      giving labels 1..N in that sequence, where N = n(n+1)/2.

    Orientation (CCW for each small triangle):
      - diagonal: (r,c) -> (r+1,c+1)               
      - left:  (r,c) -> (r, c-1)              
      - up: (r,c)   -> (r-1,c)         

    If frozen is non-empty, remove edges between the vertices in frozen. 
    """

    if not (isinstance(n, (Integer, int)) and n > 0): 
        raise ValueError("'n' must be a positive integer.") 
    if not set(F).issubset(range(1,n*(n+1)/2+1)): 
        raise ValueError("'F' must be a list consisting of elements in {1,2,...,n*(n+1)/2}.")

    # 1) Desired labeling order: bottom row to top row, each row left→right
    RC = [0]
    for c in range(1,n+1):          # c = 1, 2, ..., n  (left to right)
        for r in range(1, c+1):        # top to bottom within the column
            RC.append((r, c))
    N = len(RC)
    # Map (r,c) -> 0-based index in this labeling
    idx = {rc: i for i, rc in enumerate(RC)}

    # 2) Build edge list under CCW rule
    edges = []
    for c in range(1, n+1):
        for r in range(1, c+1):
            if c < n:
                # diagonal: (r,c) -> (r+1,c+1) 
                edges.append((idx[(r, c)], idx[(r+1, c+1)]))

            if r > 1: 
                # up: (r,c) -> (r-1,c)
                edges.append((idx[(r, c)], idx[(r-1, c)]))

            if c > r: 
                # left: (r,c) -> (r, c-1)  
                edges.append((idx[(r, c)], idx[(r, c-1)]))


    ## consider frozen vertices 
    edges_revised = edges
    for edge in edges_revised: 
        if set([edge[0], edge[1]]).issubset(set(F)):
            edges_revised.remove(edge)
        
    return ClusterQuiver(DiGraph(edges_revised), frozen=F)

## Polytope transformation (polytope mutation) 

INPUT: 
- ``P`` -- a polytope of dimension $n$
- ``Quiver`` -- ClusterQuiver having $n$ vertices
- ``k`` -- an index in $\{1,\dots,n\}$

OUTPUT:
- a Polyhedron after applying the tropicalized cluster mutation on ``P`` 

References </br>
N. Fujita and H. Oya_Newton-Okounkov polytopes of Schubert varieties arising from cluster structures,
Trans. Amer. Math. Soc. Series B 12, 910–973 (2025).

In [4]:
### 25-09-22 EJ changed the order (P, k, A) to (P, A, k) to match it with trans_seq.
## Check: We have to assume that Quiver does not have frozen vertices! 
def trans(P, Quiver, k):
    if not isinstance(P, sage.geometry.polyhedron.base.Polyhedron_base):
        raise ValueError("'P' must be a polyhedron.") 
    if not isinstance(Quiver, sage.combinat.cluster_algebra_quiver.quiver.ClusterQuiver):
        raise ValueError("'Q' must be a cluster quiver.") 

    n = P.dimension()
    A = Quiver.b_matrix() 

    if not k in range(1,n+1):
        raise ValueError("'k' must be in [1,...,n].") 

    if not n == A.nrows():
        raise ValueError("The number of vertices of 'Q' must be the dimension of 'P'.") 

    
    k = k - 1  # Convert to 0-based index
    ek = [0] * n
    ek[k] = 1  # Standard basis vector at index k
    
    # Get inequalities from the H-representation (outer normals)
    P_ineq = [[1] + list(-f.outer_normal()) for f in P.Hrepresentation()]
    
    # Cut the polytope into two parts along the hyperplane g_k = 0
    P_1 = Polyhedron(ieqs = P_ineq + [[0] + ek])
    P_2 = Polyhedron(ieqs = P_ineq + [[0] + list(-vector(ek))])
    
    New_points = []
    
    # Apply plus transformation to one part
    for v in P_1.vertices():
        New_points.append(list(matrix(v) * plus_trans(A, k + 1))[0])
    
    # Apply minus transformation to the other part
    for v in P_2.vertices():
        New_points.append(list(matrix(v) * minus_trans(A, k + 1))[0])
    
    # Return the new polytope generated by transformed vertices
    return Polyhedron(vertices = New_points)
    
def extend_square(A):
    nrows, ncols = A.nrows(), A.ncols()

    ## Since Q might have frozen vertices, the matrix A might not be a square matrix. 
    ## In this case, we have to make a square matrix! 

    if nrows != ncols: 
        A_append = matrix(nrows, nrows) 
        for i in range(nrows): 
            for j in range(nrows): 
                if j < ncols: 
                    A_append[i,j] = A[i,j]
                else: 
                    if i < ncols:
                        A_append[i,j] = -A[j,i]
                    else: 
                        A_append[i,j] = 0
        A = A_append
    
    return A

def plus_trans(A, k):
    nrows, ncols = A.nrows(), A.ncols()

    ## Since Q might have frozen vertices, the matrix A might not be a square matrix. 
    ## In this case, we have to make a square matrix! 

   
    if nrows != ncols: 
        A = extend_square(A)
    
    new_trans = matrix(nrows, nrows)
    for i in range(nrows):
        for j in range(ncols):
            if i != k - 1:
                if i != j:
                    if j == k - 1:
                        new_trans[i, j] = max(-A[k - 1, i], 0) + A[k - 1, i]
                    else:
                        new_trans[i, j] = 0
                else:
                    new_trans[i, j] = 1
            else:
                if i == j:
                    new_trans[i, j] = -1
                else:
                    new_trans[i, j] = 0
    
    return new_trans.transpose()


def minus_trans(A, k):
    nrows, ncols = A.nrows(), A.ncols()

    if nrows != ncols: 
        A = extend_square(A)
        
    new_trans = matrix(nrows, nrows)
    
    for i in range(nrows):
        for j in range(ncols):
            if i != k - 1:
                if i != j:
                    if j == k - 1:
                        new_trans[i, j] = max(-A[k - 1, i], 0)
                    else:
                        new_trans[i, j] = 0
                else:
                    new_trans[i, j] = 1
            else:
                if i == j:
                    new_trans[i, j] = -1
                else:
                    new_trans[i, j] = 0
    
    return new_trans.transpose()

In [5]:
def trans_seq(P, Q, k_list):
    new_poly = P
    new_quiver = Q
    
    for idx, k in enumerate(k_list):  # 'k' is the mutation index at each step
        if idx != 0:
            # Apply mutation to the quiver (adjacency matrix) for each step after the first
            new_quiver = new_quiver.mutate(k_list[idx - 1], inplace = False)
            # mutation(new_quiver, k_list[idx - 1])
        # Perform the transformation of the polytope with the current mutation index 'k'
        new_poly = trans(new_poly, new_quiver, k)
    
    return new_poly  # Return the final transformed polytope

## Function to <font color=red>find all possible seeds</font> that can be obtained by mutation
   We want to find all possible quivers can be obtained from E
   
#### INPUT:
   - ``E`` -- initial quiver represented as an adjacency matrix  
   - ``rank`` -- the rank of the quiver, that is, the number of mutable vertices
   
#### OUTPUT:
   - ``result`` -- list of all possible quiver and cluster variable pairs (seeds) obtained from mutation

Comparing this with 

    cluster_class_iter(depth=+Infinity, show_depth=False, up_to_equivalence=True)

we can obtain a sequence of mutations to reach each seed. 

In [6]:
def find_all_seeds(Q, frozen = None):
    ## Q is a ClusterQuiver 
    ## frozen is the frozen vertices: subset of Q.digraph().vertices() 
    ## If Q encodes the data of frozen vertices, then one could ignore it. 
    
    if not isinstance(Q, sage.combinat.cluster_algebra_quiver.quiver.ClusterQuiver):
        raise ValueError("'Q' must be a cluster quiver.") 

    if frozen: 
        froz = frozen
        mutable = [v for v in Q.free_vertices() if v not in frozen]
    else: 
        froz = Q.frozen_vertices()    
        mutable = Q.free_vertices()
    
    # Initialize result and new_quivers lists with the initial quiver and cluster variables
    S_initial = ClusterSeed(Q, frozen= froz, is_principal=False, user_labels=Q.digraph().vertices(), user_labels_prefix='x') 
    result = [[[], S_initial]]
    new_quivers = [[[], S_initial]]
    
    compare = 1  # Initialize the comparison value (to continue mutation if new quivers are found)
    
    while compare > 0:  # Continue until no new quivers are found
        temp = []  # Temporary list to store newly found seeds
        
        # Loop over all the new quivers generated in the previous iteration
        for [seq, S_initial] in new_quivers:
            # Apply mutation for each mutable vertex (1 to rank)
            quiver = S_initial.b_matrix()
            cluster = S_initial.cluster()
            for ii in mutable:
                if len(seq) == 0 or (len(seq) > 0 and ii != seq[-1]):  # Ensure no immediate repetition in mutation sequence
                    quiver_mutation_ii = S_initial.mutate(ii, input_type = 'vertices', inplace = False)
                     # Apply mutation to quiver and cluster
                    
                    # Check if the new seed is already present in the result to avoid duplicates
                    for Q in result + temp:
                        if compare_two_clusters(quiver_mutation_ii.cluster(), Q[1].cluster()):
                            break
                    else:
                        temp.append([seq + [ii], quiver_mutation_ii])  # Add new valid seed to temp list
                        
        print(f"New quivers: {len(temp)}")  # Print how many new quivers (seeds) were found
        result = result + temp  # Add new seeds to the result
        new_quivers = temp  # Update the new_quivers list with newly found quivers
        compare = len(temp)  # Set comparison to the number of new seeds found
        
    return result  # Return the list of all found seeds

In [7]:
def compare_two_clusters(cluster_1, cluster_2):
    return all([any([elt_2 == elt_1 for elt_1 in cluster_1]) for elt_2 in cluster_2])